# Training

## Imports and settings

In [ ]:
import wandb 
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from pathlib import Path
from sklearn.metrics import f1_score, accuracy_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from tqdm.notebook import tqdm 
from torch.utils.data import Dataset, DataLoader

from huggingface_hub import login

Чтобы логгировать результаты модели в wandb необходимо передать свой API ключ. Давать его в коде не лучая практика, можно тянуть из конфига, но на кагле было немного лениво этим заморачиваться. 

Доступ к модели Mistral выдается после принятия лицензионного соглашения, поэтому также понадобится ключи и от HuggingFace

In [ ]:
# Фиксируем сиды
random.seed(1337)
np.random.seed(1337)

In [ ]:
wandb.login(key='YOUR_WANDB_API_KEY')
login(token = 'YOUR_HF_API_KEY')

Для удобства задаем путь к папке с данными. Оставил тут как это было у меня в Kaggle. 

In [ ]:
DATA_PATH = Path('/kaggle/input/dls-nlp-workshop')

## Data Preprocessing

In [ ]:
with open(DATA_PATH / 'train.csv.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

random.shuffle(data)

idx, text, labels = list(zip(*[(int(row[0]), f'tags: {row[3].strip()}\n\nReview: {row[4].strip()}', row[5:]) for row in data]))
labels = np.array(labels, dtype=int)

# Веса классов в теории могут помочь при дисбалансе 
label_weights = 1 - labels.sum(axis=0) / labels.sum()

row_ids = np.arange(len(labels))
train_idx, y_train, val_idx, y_val = iterative_train_test_split(row_ids[:,np.newaxis], labels, test_size = 0.1)
x_train = [text[i] for i in train_idx.flatten()]
x_val = [text[i] for i in val_idx.flatten()]

ds = DatasetDict({
    'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
    'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
})

text[0]

In [ ]:
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

In [ ]:
model_name = 'mistralai/Mistral-7B-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

# Квантизуем в 4бит по гайду
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# Параметры конфига из гайда 
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=labels.shape[1]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/4143 [00:00<?, ? examples/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Кастомная функция для препроцессинга батча
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

# Метрики которые будем отслеживать на валидации
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    accuracy = accuracy_score(labels, predictions > 0)
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'accuracy': accuracy
    }

In [ ]:
class CustomTrainer(Trainer):

    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['val'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    label_weights = torch.tensor(label_weights, device=model.device)
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: strangerone. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241002_003149-zdusa2sz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run multilabel_classification
wandb: ⭐️ View project at https://wandb.ai/strangerone/huggingface
wandb: 🚀 View run at https://wandb.ai/strangerone/huggingface/runs/zdusa2sz
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted,Accuracy
1,0.133300,0.057007,0.551196,0.192709,0.486737,0.354167
2,0.049300,0.043728,0.672370,0.396372,0.635028,0.456250
3,0.030900,0.041099,0.725962,0.475849,0.704479,0.512500


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_

TrainOutput(global_step=1554, training_loss=0.0696430795441263, metrics={'train_runtime': 21091.718, 'train_samples_per_second': 0.589, 'train_steps_per_second': 0.074, 'total_flos': 5.775669079444685e+16, 'train_loss': 0.0696430795441263, 'epoch': 3.0})

In [ ]:
# Сохраняем модель
peft_model_id = 'multilabel_mistral'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('multilabel_mistral/tokenizer_config.json',
 'multilabel_mistral/special_tokens_map.json',
 'multilabel_mistral/tokenizer.model',
 'multilabel_mistral/added_tokens.json',
 'multilabel_mistral/tokenizer.json')

# Inference

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme here
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# Грузим сохраненную модель
base_model_id = '/kaggle/input/mistral-3-epochs-model/multilabel_mistral'
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_id, 
    quantization_config = quantization_config,
    num_labels=50)

peft_model_id = '/kaggle/input/mistral-3-epochs-model/multilabel_mistral'
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = PeftModel.from_pretrained(base_model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.config.pad_token_id = tokenizer.pad_token_id
model.to('cuda')

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

In [ ]:
with open(DATA_PATH / '/test.csv.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

idx, text = list(zip(*[(int(row[0]), f'tags: {row[3].strip()}\n\nReview: {row[4].strip()}') for row in data]))
texts = [i for i in text]

dataset = TextDataset(texts)
batch_size = 32

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()

all_logits = []

for batch_texts in tqdm(dataloader):
    tokenized_inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    tokenized_inputs = {k: v.to('cuda') for k, v in tokenized_inputs.items()}
    
    with torch.no_grad():
        outputs = model(**tokenized_inputs).logits
        all_logits.append(outputs.cpu())

all_logits = torch.cat(all_logits, dim=0)
probs = torch.sigmoid(all_logits).numpy()

In [ ]:
predictions = []
threshold = 0.6 # ВЫБРАТЬ НА КРОСС_ВАЛИДАЦИИ

def clear(row):
    row = row[1:-1]
    row = row.replace(',', ' ')
    return row.strip()

for i in np.ndindex(probs.shape[0]):
    row_indices = np.where(probs[i] > threshold)[0]
    predictions.append(row_indices)


sub = pd.DataFrame({'index':idx, 'target':[clear(str(i)) for i in predictions]})
sub.to_csv(f'submission_mistral_{threshold}.csv', index=False)
np.savetxt(f"probs_mistral_{threshold}.csv", probs, delimiter=",")